In [11]:
# CONFIG
import os
from dotenv import load_dotenv

load_dotenv()

# Airtable configuration
AIRTABLE_API_KEY = os.getenv('AIRTABLE_API_KEY')
AIRTABLE_BASE_ID = os.getenv('AIRTABLE_BASE_ID')
AIRTABLE_CD_TABLE = os.getenv('AIRTABLE_CD_TABLE')

# Data Retrieval

In [13]:
import pandas as pd
from pyairtable import Api

# data processing
def remove_fields_prefix(df):
    df.columns = df.columns.str.replace('fields.', '')
    return df

def process_list_like_values(df):
    for col in df.columns:
        if df[col].apply(lambda x: isinstance(x, list)).any():
            df[col] = df[col].apply(lambda x: ','.join(map(str, x)) if isinstance(x, list) else x)
    return df

# data fetching
def fetch_data(table_id):
    api = Api(AIRTABLE_API_KEY)
    table = api.table(AIRTABLE_BASE_ID, table_id)
    records = table.all()

    if records:
        df = pd.json_normalize([record['fields'] for record in records])  # Directly normalize 'fields'
        df = remove_fields_prefix(df)
        df = process_list_like_values(df)
        return df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no records

# implementation
df = fetch_data(AIRTABLE_CD_TABLE)

columns_to_check = [
    "Relevant facts / Summary of the case", 
    "Relevant rules of law", 
    "Choice of law issue", 
    "Court's position"
]

df = df.dropna(subset=columns_to_check)

cases = df['Case'].to_list()
abstracts = df['Abstract'].to_list()
texts = df['Content'].to_list()
facts = df['Relevant facts / Summary of the case'].to_list()
rules = df['Relevant rules of law'].to_list()
issues = df['Choice of law issue'].to_list()
positions = df["Court's position"].to_list()

# Case Analysis

In [17]:
# select one single court decision

cd = texts[0]

In [14]:
#utils

import subprocess

def prompt_llama(prompt_text):
    # Pass the prompt text directly as input without flags
    process = subprocess.run(
        ["ollama", "run", "llama3.1"],
        input=prompt_text,  # Provide the prompt directly
        capture_output=True,
        text=True
    )

    # Handle errors if they occur
    if process.returncode != 0:
        print("Error:", process.stderr)
        return None
    
    # Return the model's output
    return process.stdout.strip()

## 0. Abstract

In [18]:
# extract abstracts from texts
prompt_abstract = f"""Here is the text of a Court Decision:
                {cd}
                Find the official abstract stated in the case, usually right at the beginning. This task is successfully accomplished if it is a precise "copy-paste" of the Regeste paragraph of the file. This term varies according to country to country (e.g., Brazil: Ementa).  The translation of certain legal concepts can be challenging. It is a good abstract if it conveys the same meaning (not necessarily obtained via a literal translation).

                Otherwise, this has to return a general description of the information in the file. It's successfully accomplished if (i) it is concise; (ii) it manages to condense all the key details (topic, provisions, information about the legal dispute) in a single paragraph or less. 
                """
abstract_l3 = prompt_llama(prompt_abstract)
print(abstract_l3)

Here is a possible abstract based on the provided text:

"Dispute over notice of termination: The Swiss Federal Supreme Court upholds a cantonal court's decision that an employee was validly dismissed verbally during an interview with two representatives, despite her claims to the contrary and allegations of arbitrariness. The court concludes that the employee should have inferred a representative relationship from the circumstances and that the dismissal was clearly communicated."


## 1. Relevant Facts

In [19]:
# relevant facts
prompt_facts = f"""Here is the text of a Court Decision:
                {cd}
                Create a short summary of the facts of the case. Who are the parties, what happened, what is the dispute about and what are the different stages of court proceedings etc. The summary prioritizes information on choice of law.
                It's successfully accomplished when I understand the case from a private international law perspective (e.g., providing which jurisdictions are involved - be it through the parties or legal frameworks) and what happened in the case, particularly related to the choice of law issue. It should be written in an accessible manner. It should contain accurate information (cannot add assumptions that are not stated in the case!). It should be a structured paragraph meaningful for private international law practitioners.
                """
facts_l3 = prompt_llama(prompt_facts)
print(facts_l3)

The case involves a dispute between two employees, B and A, representing their manager C, who terminated the employment contract of the appellant on April 25, 2007. The termination was communicated verbally by B and A to the appellant during an interview. The appellant claims that her dismissal was arbitrary, arguing that she did not understand that B and A were acting as representatives of C.

In this case, there are no parties from different jurisdictions involved in the dispute, so private international law principles related to jurisdictional conflicts are not applicable. However, the choice of law issue is relevant because it concerns the interpretation of Swiss employment law (FSCA).

The dispute centers on whether the appellant understood that B and A were representing C when they communicated her termination. The cantonal court found that the appellant was obligated to infer a relationship of representation between B/A and C based on their conduct, including explaining reasons 

## 2. Relevant Rules of Law

In [21]:
# relevant rules of law
prompt_rules = f"""Here is the text of a Court Decision:
                {cd}
                Extract all legislations and court decisions from the given court decision. Differentiate between legislations and court decisions that have to do with the theme of Choice of Law and legislations and court decisions of a general manner.
                This category is called rules of law because (i) it is not necessarily a provision that is clearly cited - it can be the understanding of the court. (ii) it does not necessarily need to be a provision stemming from State law.
                """
rules_l3 = prompt_llama(prompt_rules)
print(rules_l3)

**Legislations:**

1. **Federal Act on Civil Procedure (FSCA)**:
	* Article 105, paragraph 1
	* Article 66, article 1 and 2
	* Article 68, articles 1 and 2
2. **Swiss Code of Obligations (CO)**:
	* Article 335 CO

**Court Decisions:**

1. **Federal Supreme Court Decision**:
	* BGE 128 III 129 (recital 2b, p. 135)
	* 4C.151/2003 (August 26, 2008) - para. 4.3
2. **Rules of Law**:
	* The understanding of the court regarding the principle that a person receiving a notice must unambiguously understand the meaning of the declaration.
	* The notion that the spirit behind an employment termination is very similar to the one that led to hiring, implying that the employee should infer a representative relationship.


## 3. Choice of Law Issue

In [25]:
# choice of law issue
prompt_issue_theme = f"""Here is the text of a Court Decision:
                {cd}
                Classify it into one main Choice of Law concept and return only the concept. NO EXPLANATION WHATSOEVER.
                """
issue_theme_l3 = prompt_llama(prompt_issue_theme)
print(issue_theme_l3)

Contract Law


In [27]:
#issue_theme_l3
theme_definition = "this is a mock definition"

In [28]:
# choice of law issue
prompt_issue = f"""Here is the text of a Court Decision:
                {cd}
                The concept {issue_concept_l3} has been identified to be related with this court decision. The concept is defined as follows:
                {theme_definition}
                Refer to the main - private international law - problem, and phrase it in the format of a concise question. The issue should overlap with the classified theme - this should be of assistance to show what “category” of choice of law the case deals with. 
                This is successfully accomplished when it can be boiled down to a yes/no answer. Alternatively, it can also be a descriptive text talking about the questions in regard to choice of law that are answered by the specific case. 
                """
issue_l3 = prompt_llama(prompt_issue)
print(issue_l3)

Here's a concise question that addresses the issue:

**Can an employee infer a representative relationship between their manager and colleagues who communicate their dismissal, making them responsible for giving notice?**

This question overlaps with the theme "Agency/Representation" in employment contracts, specifically addressing issues of vicarious liability and representation in dismissal proceedings.


## 4. Court's Position

In [29]:
# court's position
prompt_position = f"""Here is the text of a Court Decision:
                {cd}
                Extract the court’s position in regard to the statements made in the “choice of law issue” tab. It can be phrased as an answer to the question or as a descriptive text. It’s important that the answers correlate with the answers provided in the “answers” tab (confirming, refuting, or adding more information to something).
                This is successfully accomplished when I understand the answer (yes/no/depends on certain circumstances outlined by the specific context of the case) related to the previous problem (Choice of Law Issue).
                """
position_l3 = prompt_llama(prompt_position)
print(position_l3)

Here's a descriptive text that summarizes the court's position in regard to the statements made in the "choice of law issue" tab:

The court confirms that the dismissal was effectively communicated to the appellant, and therefore, Swiss federal law is applicable. The court finds no fault with the communication process, as it was verbally conveyed by the witnesses present at the interview on April 25, 2007, which the appellant herself admitted in an email sent to her representative on May 14, 2007.

In other words, the answer to the "choice of law issue" tab is a confirmation that Swiss federal law applies.
